# Multiple Dispatch

### Basic dispatch

In [11]:
f(a, b::Any) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [12]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[11]:5
[2] f(a::Number, b::Number) in Main at In[11]:2
[3] f(a::Number, b) in Main at In[11]:3
[4] f(a, b::Number) in Main at In[11]:4
[5] f(a, b) in Main at In[11]:1

In [13]:
f(1.5, 2)

"a and b are both numbers"

In [14]:
f(1, "bar")

"a is a number"

In [15]:
f(1, 2)

"a and b are both integers"

In [16]:
f("foo", [1,2])

"fallback"

In [17]:
f(1, 2, 3)

MethodError: MethodError: no method matching f(::Int64, ::Int64, ::Int64)
Closest candidates are:
  f(::Integer, ::Integer) at In[11]:5
  f(::Number, ::Number) at In[11]:2
  f(::Number, ::Any) at In[11]:3
  ...

### Ambiguities

In [18]:
g(a::Int, b::Number) = 1
g(a::Number, b::Int) = 2

g (generic function with 2 methods)

In [19]:
g(1, 2.5)

1

In [20]:
g(1.5, 2)

2

In [21]:
g(1, 2)

MethodError: MethodError: g(::Int64, ::Int64) is ambiguous. Candidates:
  g(a::Number, b::Int64) in Main at In[18]:2
  g(a::Int64, b::Number) in Main at In[18]:1
Possible fix, define
  g(::Int64, ::Int64)

In [22]:
g(x::Int, y::Int) = 3

g (generic function with 3 methods)

In [23]:
g(1, 2)

3

### "Diagonal" dispatch

In [24]:
f(a::T, b::T) where {T<:Number} = "a and b are both $(T)s"

f (generic function with 6 methods)

In [25]:
methods(f)

# 6 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[11]:5
[2] f(a::T, b::T) where T<:Number in Main at In[24]:1
[3] f(a::Number, b::Number) in Main at In[11]:2
[4] f(a::Number, b) in Main at In[11]:3
[5] f(a, b::Number) in Main at In[11]:4
[6] f(a, b) in Main at In[11]:1

In [26]:
f(big(1.5), big(2.5))

"a and b are both BigFloats"

In [27]:
f(big(1), big(2)) # <== integer rule is more specific

"a and b are both integers"

In [28]:
f(a::T, b::T) where {T<:Integer} = "both are $T integers"

f (generic function with 7 methods)

In [29]:
methods(f)

# 7 methods for generic function "f":
[1] f(a::T, b::T) where T<:Integer in Main at In[28]:1
[2] f(a::Integer, b::Integer) in Main at In[11]:5
[3] f(a::T, b::T) where T<:Number in Main at In[24]:1
[4] f(a::Number, b::Number) in Main at In[11]:2
[5] f(a::Number, b) in Main at In[11]:3
[6] f(a, b::Number) in Main at In[11]:4
[7] f(a, b) in Main at In[11]:1

In [30]:
f(big(1), big(2))

"both are BigInt integers"

In [31]:
f(1, big(2))

"a and b are both integers"

In [32]:
f("foo", "bar") # <== still doesn't apply to non-numbers

"fallback"

### Varargs methods

In [38]:
v(args...) = args

v (generic function with 1 method)

In [39]:
v(1, "foo", 12.5)

(1, "foo", 12.5)

In [34]:
f(args::Number...) = "$(length(args))-ary heterogeneous call"
f(args::T...) where {T<:Number} = "$(length(args))-ary homogeneous call"

f (generic function with 9 methods)

In [35]:
methods(f)

# 9 methods for generic function "f":
[1] f(a::T, b::T) where T<:Integer in Main at In[28]:1
[2] f(a::Integer, b::Integer) in Main at In[11]:5
[3] f(a::T, b::T) where T<:Number in Main at In[24]:1
[4] f(a::Number, b::Number) in Main at In[11]:2
[5] f(args::T...) where T<:Number in Main at In[34]:2
[6] f(args::Number...) in Main at In[34]:1
[7] f(a::Number, b) in Main at In[11]:3
[8] f(a, b::Number) in Main at In[11]:4
[9] f(a, b) in Main at In[11]:1

In [36]:
f(1)

"1-ary homogeneous call"

In [37]:
f(1, 2, 3)

"3-ary homogeneous call"

In [40]:
f(1, 1.5, 2)

"3-ary heterogeneous call"

In [41]:
f()

"0-ary homogeneous call"

In [42]:
f(1, 2) # <== previous 2-arg method is more specific

"both are Int64 integers"

In [43]:
f("foo") # <== still doesn't apply to non-numbers

MethodError: MethodError: no method matching f(::String)
Closest candidates are:
  f(::Any, !Matched::Number) at In[11]:4
  f(::Any, !Matched::Any) at In[11]:1
  f(!Matched::T<:Integer, !Matched::T<:Integer) where T<:Integer at In[28]:1
  ...

In [44]:
# "splat" (more below)
f([1, 2, 3]...)

"3-ary homogeneous call"

In [46]:
v([1, 2, 3])

([1, 2, 3],)

In [47]:
v([1, 2, 3]...)

(1, 2, 3)

In [48]:
v(1, 2, 3) # equivalent to this

(1, 2, 3)

In [55]:
a = rand(100)
+(a...) # don't do this!
sum(a) # do this inistead
reduce(+, a) # or do this

51.1022532586601

### Optional Arguments

In [56]:
h(x, y = 0) = 2x + 3y

h (generic function with 2 methods)

In [57]:
methods(h)

# 2 methods for generic function "h":
[1] h(x) in Main at In[56]:1
[2] h(x, y) in Main at In[56]:1

Shorthand for this:
```jl
h(x, y) = 2x + 3y
h(x) = h(x, 0)
```

In [58]:
m(a=0, b=1, c=2, d=3, e=4) = a + b + c + d + e

m (generic function with 6 methods)

In [59]:
methods(m)

# 6 methods for generic function "m":
[1] m() in Main at In[58]:1
[2] m(a) in Main at In[58]:1
[3] m(a, b) in Main at In[58]:1
[4] m(a, b, c) in Main at In[58]:1
[5] m(a, b, c, d) in Main at In[58]:1
[6] m(a, b, c, d, e) in Main at In[58]:1

In [62]:
m(2, 3)

14

### Keyword Arguments

In [63]:
k(x, y = 0; opt::Bool = false) = opt ? 2x+y : x+2y

k (generic function with 2 methods)

In [64]:
methods(k)

# 2 methods for generic function "k":
[1] k(x) in Main at In[63]:1
[2] k(x, y; opt) in Main at In[63]:1

In [65]:
k(2)

2

In [66]:
k(2, 3)

8

In [67]:
k(2, opt=true)

4

In [68]:
k(2, 3, opt=true)

7

In [67]:
k(2, opt=false)

4

In [70]:
k(2, opt=false, 3)

8

In [71]:
k(opt=true, 2, 3)

7

In [72]:
foo(x, y; req::Bool) = req ? 2x+y : x+2y

foo (generic function with 1 method)

In [73]:
foo(2, 3)

UndefKeywordError: UndefKeywordError: keyword argument req not assigned

In [74]:
foo(2, 3, req=true)

7

In [75]:
foo(2, 3, req=false)

8

In [78]:
function takes_lots_of_options(
    arg1::Int,
    arg2::Int; # semicolon marks end of positional args
    frobble::Bool = false,
    bias::Float64 = 2.0,
    level::Int = 11,
)
    out = (arg1^bias + arg2^bias)^(1/bias) + level
    return frobble ? -out : out
end

takes_lots_of_options (generic function with 2 methods)

In [79]:
takes_lots_of_options(1, 2, level = -5, bias = 1.5)

-2.5527391852285244

### Keyword arguments: slurp and splat

In [82]:
function allkw(; kw...)
    @show kw
end

allkw (generic function with 1 method)

In [83]:
allkw(a=1,b=2)

kw = Base.Iterators.Pairs(:a=>1,:b=>2)


pairs(::NamedTuple) with 2 entries:
  :a => 1
  :b => 2

Just like iterators can be splatted as positional arguments, dict-like collections and named tuples can be splatted as keyword arguments.

In [104]:
function rect(; width=1, height=1, fill="#")
    for i in 1:height
        println(fill^width)
    end
end

rect (generic function with 1 method)

In [97]:
params = (width=8, height=3, fill="X")

(width = 8, height = 3, fill = "X")

In [98]:
typeof(params)

NamedTuple{(:width, :height, :fill),Tuple{Int64,Int64,String}}

In [105]:
rect(; params..., fill=99)

9227446944279201
9227446944279201
9227446944279201


### Exercises

#### Exercise 1

Write a function, `repstr`, that repeats a string an integer number of times which takes the arguments in either order. You can call the built-in `^` operator that repeats a string.

In [106]:
repstr(str::String, rep::Int) = str^rep
repstr(rep::Int, str::String) = str^rep

repstr (generic function with 2 methods)

In [108]:
repstr(str::String, rep::Int) = str^rep
repstr(rep::Int, str::String) = repstr(str, rep)

repstr (generic function with 2 methods)

In [109]:
@assert repstr("abc", 3) == "abcabcabc"
@assert repstr(3, "abc") == "abcabcabc"

In [110]:
repstrk(; str::String, rep::Int) = str^rep

repstrk (generic function with 1 method)

@assert repstrk(str="abc", rep=3) == "abcabcabc"
@assert repstrk(rep=3, str="abc") == "abcabcabc"

#### Exercise 2a

Write a function `F` that returns the tuple `(x, y, k)` where:
- `x` is the first positional argument and is mandatory
- `y` is the second positional argument and is optional
- `k` is an optional keyword argument

The optional arguments should have the following defaults:
- `y` defaults to `2x`
- `k` defaults to `2y`

In [112]:
function ex2a(x, y=2x; k=2y)
    return (x, y, k)
end

ex2a (generic function with 2 methods)

In [113]:
ex2a(2)

(2, 4, 8)

In [114]:
ex2a(2, 5)

(2, 5, 10)

In [115]:
ex2a(3, k=1)

(3, 6, 1)

In [117]:
ex2a(x, y=2x; k=2y) = (x, y, k) # equivalent

ex2a (generic function with 2 methods)

#### Exercise 2b

Write a function `G` just like `F` but with differet defaults:
- `k` defaults to `2x`
- `y` defaults to `2k`

In [131]:
G(x, y::Number=false; k=2x) = (x, (y == false ? 2k : y), k)

G (generic function with 3 methods)

In [140]:
G(x, y=nothing; k=2x) = (x, (y == nothing ? 2k : y), k)

G (generic function with 3 methods)

In [146]:
G(x, y; k=2x) = (x,  y, k)
G(x;    k=2x) = (x, 2k, k)

G (generic function with 3 methods)

In [147]:
G(1)

(1, 4, 2)

In [148]:
G(1, 3)

(1, 3, 2)

In [149]:
G(1, k=5)

(1, 10, 5)

In [150]:
G(2, 3, k=5)

(2, 3, 5)

In [151]:
G(1, "foo")

(1, "foo", 2)

### Side discussion on booleans being numbers

In [132]:
true::Number

true

In [135]:
false*123

0

In [136]:
true*123

123

In [137]:
typeof(im)

Complex{Bool}

In [138]:
im

im

In [139]:
dump(im)

Complex{Bool}
  re: Bool false
  im: Bool true
